In [74]:
# Import packages

from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np
import spacy
import nltk
nltk.download('punkt')
spacy = spacy.load('en_core_web_sm')
!pip install --quiet pytorch-lightning
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
# Import packages
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
!pip install transformers
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )
import re

from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
# DOWNLOADING TENSORFLOW LIBRARIES TO EASILY DOWNLOAD THE POPULAR SENTENCE
# EMBEDDING TECHNIQUE - UNIVERSAL SENTENCE ENCODER


pl.seed_everything(20)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Global seed set to 20


20

# Introduction

In this notebook we combined all 3 of our trained models to create a clear pipeline of our system.

Requirements:
1. weigths files for 3 of our models (Answer Gen, Question Gen and Distractors Gen)
2. The Json files of Squad1.1 if you decide to make predictions on them (train-v1.1.json, dev-v1.1.json)


Connecting to Google drive to get the weights for the 3 models

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# LoadUp the data from squad1.1

We can create answers,questions and distractors from this database or just provide our own text like will be displayed later down the notebook

In [76]:
train = pd.read_json('train-v1.1.json')
dev = pd.read_json('dev-v1.1.json')


# Step 1 Generate 1-10 answers per context
Stav's Model

Converting the JSON files to dataframes.

In [77]:
def DataDFfromSquad(dataset):
    
  data=[]
  for article in (dataset['data']):
    title=article['title']
    for paragraph in article['paragraphs']:
      answers=[]
      para=paragraph['context']
      for i in range(len(paragraph['qas'])):
         answers.append(paragraph['qas'][i]['answers'][0]['text'])
      data.append([para,answers])
  dataframe=pd.DataFrame(data=data,columns=['context','answers'])

  return dataframe

train_df=DataDFfromSquad(train)
val_df=DataDFfromSquad(dev)
train_df = train_df.sample(frac=1,random_state=20).reset_index(drop=True)
test_df = train_df[:1000] 
train_df = train_df[1000:]


In [78]:
train_df.head()

,context,answers
1000,An early advantage of AC is that the power-was...,"[multiple taps on the transformer, power-wasti..."
1001,"""Green"" in modern European languages correspon...","[Thai, 520–570 nm, the name of a fruit]"
1002,Almost all Protestant denominations are repres...,"[74, two, 1814, Saint George's Anglican Church]"
1003,Hurling and football are the most popular spec...,"[Hurling and football, 30, 7, Camogie, gaelic ..."
1004,"From 1932, it was taught that the ""little floc...","[1932, 144,000, a paradise restored on earth, ..."


# Load the AnswerGen model


In [79]:
class AGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
        self.model.resize_token_embeddings(32102) 

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)

Loading up the AnswerGen Weights

In [80]:
checkpoint_path = '/content/drive/MyDrive/AnswerGen/AnswerGenWeights.ckpt'
AnswerGen_model = AGModel.load_from_checkpoint(checkpoint_path)
AnswerGen_model.freeze()
AnswerGen_model.eval()
print()

# Generating function for the model 

In [124]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
SEP_TOKEN='<<sep>>'
ANS_TOKEN='<<ans>>'
tokenizer.add_tokens(SEP_TOKEN)
tokenizer.add_tokens(ANS_TOKEN)

def generate_answerNoraml(AGModel: AGModel ,context,max_length,num_beams) : # The first Function to generate up to 10 answers per context
    SEP_TOKEN='<<sep>>'
    ANS_TOKEN='<<ans>>'
    string=""
    for i in range(10): # stringing together 10 Mask tokens
      string+=' '
      string+='@MASK@'
      string+=' '
      string+=ANS_TOKEN
    string+=SEP_TOKEN
    string+=' '
    string+=context
    source_encoding = tokenizer(
        string,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = AGModel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=10,
        max_length=250,
        repetition_penalty=5.0,
        no_repeat_ngram_size=3,
        length_penalty=5.0,
        early_stopping=True,
        use_cache=True
    )
    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }
    return ''.join(preds)
    
def generate_answerWithMinLength(AGModel: AGModel ,context,max_length,num_beams,min_length) : # the second function that will be used to generate an increased amount of answers per text
    SEP_TOKEN='<<sep>>'
    ANS_TOKEN='<<ans>>'
    string=""
    for i in range(10): # stringing together 10 Mask tokens
      string+=' '
      string+='@MASK@'
      string+=' '
      string+=ANS_TOKEN
    string+=SEP_TOKEN
    string+=' '
    string+=context
    source_encoding = tokenizer(
        string,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = AGModel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=num_beams,
        max_length=250,
        min_length=min_length, # using minimum length to force the model into generating more answers
        repetition_penalty=5.0,
        no_repeat_ngram_size=3,
        length_penalty=5.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)
def GeneAnswersToListNormal(best_model,context,len,num_beams): # an helper function to remove tokens that were created during the generation procces
  ans=generate_answerNoraml(best_model,context,len,num_beams)
  ans=ans.replace('<pad>','')
  answer=''
  answers=[]
  flag=0
  for word in ans:
    if word=='<':
      flag=1
    elif word =='>':
      flag=0
      if answer!='':
        answers.append(answer)
        answer=''
    else:
      if flag==0:
        answer=answer + word
  return answers
  
def GeneAnswersToListMinLength(best_model,context,len,num_beams,min_length):#an helper function to remove tokens that were created during the generation procces this time using the min_length parameter
  ans=generate_answerWithMinLength(best_model,context,len,num_beams,min_length)
  ans=ans.replace('<pad>','')
  answer=''
  answers=[]
  flag=0
  for word in ans:
    if word=='<':
      flag=1
    elif word =='>':
      flag=0
      if answer!='':
        answers.append(answer)
        answer=''
    else:
      if flag==0:
        answer=answer + word
  return answers



def GeneAnswersForDFToDF(best_model,dataset,max_length,frac=0.15): # create a Dataframe of [context, answers] pairs
  
  answersSet=set()
  answersCheckDuplicateSet=set()
  contextAnswerList=[]
  dataset = dataset.sample(frac=frac,random_state=20).reset_index(drop=True)
  
  for context in dataset['context']:
    num_beams=10
    answersList=GeneAnswersToListNormal(best_model,context,max_length,num_beams) #generate up to 10 answers per context
    answersListOrginial=answersList.copy()
    answersSet=set()
    answersCheckDuplicateSet=set()
    for i in range(len(answersList)):  #making sure our answers are unique and we are not getting double answers like "The Cat", "the cat", "The cat."
          answersList[i]=answersList[i].lower()
          answersList[i] = re.sub("""[!@#()-+$&^]['":,.]""", '', answersList[i])
          if answersList[i] not in answersCheckDuplicateSet and answersList[i]!="mask":
            answersSet.add(answersListOrginial[i])
          answersCheckDuplicateSet.add(answersList[i])

    for answer in answersSet:
        contextAnswerList.append([context,answer])
  dataframe=pd.DataFrame(data=contextAnswerList,columns=['context','answer'])
  return dataframe




def GeneAnswersForDFToDFWithMaxAnswers(best_model,dataset,max_length,max_answers,frac=0.15): # create a Dataframe of [context, answers] pairs

  contextAnswerList=[]
  dataset = dataset.sample(frac=frac,random_state=20).reset_index(drop=True)
  for context in dataset['context']:
    LessThanXFlag=True
    min_length=10
    num_beams=10
    firstTime=True
    while LessThanXFlag:
      if firstTime:
        answersSet=set()
        answersCheckDuplicateSet=set()
        firsTime=False

      answersList=GeneAnswersToListMinLength(best_model,context,max_length,num_beams,min_length) #generate Answers with a required amount of answers
      answersListOrginial=answersList.copy()
      
      for i in range(len(answersList)): #making sure our answers are unique and we are not getting double answers like "The Cat", "the cat", "The cat."
          answersList[i]=answersList[i].lower()
          answersList[i] = re.sub("""[!@#()-+$&^]['":,.]""", '', answersList[i])
          if answersList[i] not in answersCheckDuplicateSet and answersList[i]!="mask":
            answersSet.add(answersListOrginial[i])
            if len(answersSet)==max_answers:
               LessThanXFlag=False
               break
          answersCheckDuplicateSet.add(answersList[i])
        
      if len(answersSet)==max_answers:
          LessThanXFlag=False
      else:
        min_length+=40

    for answer in answersSet:
        contextAnswerList.append([context,answer])
  dataframe=pd.DataFrame(data=contextAnswerList,columns=['context','answer'])
  return dataframe    
    
      


# Generate exactly 10 answers per context from Database

This function can be used to generate more than 10 answers by changing the parameter "max_answers"

In [ ]:
max_answers=10
contextAnswer_df=GeneAnswersForDFToDFWithMaxAnswers(AnswerGen_model,test_df,500,max_answers)
contextAnswer_df.to_csv('contextAnswerNormal_df.csv',encoding='utf-8', index=False)
contextAnswer_df

# Generate Answers for Context in test_df normally, maximum 10 answers

In [82]:
contextAnswer_df=GeneAnswersForDFToDF(AnswerGen_model,test_df,500)
contextAnswer_df.to_csv('contextAnswerNormal_df.csv',encoding='utf-8', index=False)
contextAnswer_df

,context,answer
0,A much smaller number of people are insensitiv...,congenital insensitivity to pain
1,A much smaller number of people are insensitiv...,a reduced life expectancy
2,A much smaller number of people are insensitiv...,sodium channel (Nav1.7) necessary in conducti...
3,A much smaller number of people are insensitiv...,one of five hereditary sensory and autonomic ...
4,A much smaller number of people are insensitiv...,inflammation
...,...,...
1198,"However, several de facto standards for tag fo...",2010
1199,"However, several de facto standards for tag fo...",identification3v1 and ID3v2
1200,"However, several de facto standards for tag fo...",Tag format
1201,"However, several de facto standards for tag fo...","at the beginning or end of MP3 files, separat..."


# Step 2 Create Question for each Answer
Doron's Model


In [117]:
class QuesGenModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
        self.model.resize_token_embeddings(32100)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)


In [118]:
checkpoint_path = '/content/drive/MyDrive/AnswerGen/GenerateQuestionWeights.ckpt'

question_model = QuesGenModel.load_from_checkpoint(checkpoint_path)
question_model.freeze()
question_model.eval()
print()

def generate_question(answer, context, best_model):
    source_encoding = tokenizer(
        answer,
        context,
        max_length=500,
        padding='max_length',
        truncation='only_second',
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    generated_ids = best_model.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=1,
        max_length=30,
        repetition_penalty=3.5,
        length_penalty=0.95,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)

In [119]:
def ans_text(input_df): 
 
    output = {'context': [], 'answer': [], 'question': []}

    for i in range(len(input_df)):
      
        context = input_df.iloc[i]['context']
        answer = input_df.iloc[i]['answer']
        
        question = generate_question(answer, context, question_model) # Remember the definition of generated

        output['context'].append(context)
        output['answer'].append(answer)
        output['question'].append(question)

    return pd.DataFrame(output)

In [86]:
context_ans = pd.read_csv('/content/contextAnswerNormal_df.csv')
context_ans

,context,answer
0,A much smaller number of people are insensitiv...,congenital insensitivity to pain
1,A much smaller number of people are insensitiv...,a reduced life expectancy
2,A much smaller number of people are insensitiv...,sodium channel (Nav1.7) necessary in conducti...
3,A much smaller number of people are insensitiv...,one of five hereditary sensory and autonomic ...
4,A much smaller number of people are insensitiv...,inflammation
...,...,...
1198,"However, several de facto standards for tag fo...",2010
1199,"However, several de facto standards for tag fo...",identification3v1 and ID3v2
1200,"However, several de facto standards for tag fo...",Tag format
1201,"However, several de facto standards for tag fo...","at the beginning or end of MP3 files, separat..."


In [87]:
df = ans_text(context_ans)
columns_titles = ["context","question","answer"]
df=df.reindex(columns=columns_titles)

df.to_csv('ContextAnswerQuestion.csv',encoding='utf-8', index=False)

In [88]:
df

,context,question,answer
0,A much smaller number of people are insensitiv...,What is the inborn abnormality of a nervous sy...,congenital insensitivity to pain
1,A much smaller number of people are insensitiv...,What is the other side effect of congenital in...,a reduced life expectancy
2,A much smaller number of people are insensitiv...,What does the SCN9A gene code for?,sodium channel (Nav1.7) necessary in conducti...
3,A much smaller number of people are insensitiv...,What are most people with congenital insensiti...,one of five hereditary sensory and autonomic ...
4,A much smaller number of people are insensitiv...,What is the name of a condition that affects c...,inflammation
...,...,...,...
1198,"However, several de facto standards for tag fo...",When did ID3v1 and APEvi2 become more widespread?,2010
1199,"However, several de facto standards for tag fo...",What are the most widespread standards for tag...,identification3v1 and ID3v2
1200,"However, several de facto standards for tag fo...",What type of format exist in the United States?,Tag format
1201,"However, several de facto standards for tag fo...",When are tags normally embedded?,"at the beginning or end of MP3 files, separat..."


# Step 3 Distractor Gen 
Avitiay Model

In [89]:
# CREATING THE MODEL CLASS
from transformers import get_cosine_schedule_with_warmup
from transformers import get_cosine_with_hard_restarts_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup
import math

MODEL_NAME = 't5-small'
N_EPOCHS = 15
SOURCE_MAX_TOKEN_LEN = 496
TARGET_MAX_TOKEN_LEN = 54
BATCH_SIZE = 16
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.001
WARMUP = 0.2
SEP_TOKEN = '<SEP>'
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens(SEP_TOKEN) # ADDING THE SEPERATION TOKEN TO THE TOKENIZER


# CREATING THE MODEL CLASS

class SAD_Model(pl.LightningModule):
  
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
    self.model.resize_token_embeddings(len(tokenizer)) # SEPERATION TOKEN WAS ADDED SO RESIZING IS REQUIRED

  def forward(self, input_ids, attention_mask, labels = None):
    output = self.model(input_ids = input_ids, attention_mask = attention_mask, labels = labels)
    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY) # CONFIGURING ADAMW OPTIMIZER WITH WEIGHT DECAY
    training_steps = TRAIN_SIZE/BATCH_SIZE
    warmup_steps = math.floor(training_steps * WARMUP) # NUMBER OF WARMUP STEPS FOR THE SCHEDULER
    #scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, training_steps)
    scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, warmup_steps, training_steps) # DEFINING THE SCHEDULER FOR THE LEARNING RATE
    #scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, training_steps)
    return [optimizer],[scheduler]

In [90]:
# LOADING THE BEST MODEL WE ACHIEVED DURING TRAINING

distractor_model = SAD_Model.load_from_checkpoint("/content/drive/MyDrive/AnswerGen/AvitayWeights.ckpt")

#MAKING SURE NO MORE GRADIENT CALCULCATIONS TAKE PLACE:
distractor_model.freeze()
distractor_model.eval()
print('')

# CONFIGURING PARAMETERS FOR OUR MODEL


In [91]:
# GENERATING OUR FALSE ANSWERS
def generate_preds(trained_model: SAD_Model, answer, context, question, num_of_generations):
    source_encoding = tokenizer(
        answer+SEP_TOKEN+question+SEP_TOKEN+context,  # INPUT: ANSWER, QUESTION, CONTEXT
        max_length=SOURCE_MAX_TOKEN_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    generated_ids = trained_model.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_return_sequences=num_of_generations, # NUMBER OF DIFFERENT FALSE ANSWERS GENERATIONS
        max_length=TARGET_MAX_TOKEN_LEN,
        do_sample=True, # WHETHER OR NOT TO USE SAMPLING
        top_k=50, # TOP K SAMPLING
        no_repeat_ngram_size=3, # AVOIDING REPEATING 3 GRAM PHRASES IN OUR FALSE ANSWERS
        top_p=0.95, # TOP P SAMPLING
        length_penalty=0.65, # PENALIZING FOR TOO LONG FALSE ANSWERS GENERATED
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)  # A CONTINOUS STRING WITH ALL THE FALSE ANSWERS SEPERATED BY <> TOKENS IS OUTPUTTED
# FUNCTION THAT PROCESSES THE CONTINOUS STRING WHICH CONTAINS NUMEROUS FALSE
# ANSWERS INTO A LIST OF FALSE ANSWERS.

def clean_prediction(pre_processed: str): 
  pre_processed = pre_processed.replace('<pad>', '') # ELIMINATING THE PAD TOKENS FROM THE CONTINOUS STRING

  word = ''
  answers = []

  flag = False
  for char in pre_processed:
    if char == '<': # INDICATING START OF A SPECIAL TOKEN SEPERATING BETWEEN SENTENCES
      flag = True  # WHEN FLAG IS TRUE, WE ARE STILL AT A SPECIAL TOKEN THAT WE WANT TO IGNORE
    elif char == '>': # INDICATING END OF THAT TOKEN, AND ACTUALLY INDICATING THE END OF A FALSE ANSWER
      flag = False
      answers.append(word) # APPEND THE FALSE ANSWER TO THE LIST
      word = '' # RESET 
    else:
      if flag == False:
        word = word + char

  return answers  # RETURN LIST OF FALSE ANSWERS

  # DOWNLOADING AND LOADING THE UNIVERSAL SENTENCE ENCODER
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
USE_model = hub.load(module_url)
print ("module %s loaded" % module_url)

# A FUNCTION THAT CALCULATES THE COSINE SIMILARITY BETWEEN TWO VECTORS, LATER ON USED
# TO CALCULATE IT BETWEEN SENTENCE EMBEDDINGS, TO DETERMINE SIMILARITY BETWEEN SENTENCES.
def cosine(u, v):
  return np.dot(u, v) / ((np.linalg.norm(u) * np.linalg.norm(v)))

# DOWNLOADING AND LOADING GLoVE FOR LATER CALCULATIONS OF WORD SIMILARITIES
# USED FOR COMPARISON BETWEEN ANSWERS WHICH ARE OF LENGTH 1 (WORDS, AND NOT SENTENCES).
from gensim import downloader
GLOVE_PATH = 'glove-twitter-50'
glove = downloader.load(GLOVE_PATH)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [148]:
# A FUNCTION THAT GENERATES 3 FALSE ANSWERS TO EACH SAMPLE IN THE INPUTTED DATAFRAME.
# AT THE BEGINNING, THE DATAFRAME CONTAINS ONLY CONTEXT, ANSWER AND QUESTION.
# THEN WE OUTPUT THE DATAFRAME WITH 3 GENERATED FALSE ANSWERS FOR EACH SAMPLE OF THE ORIGINAL DATAFRAME.

#### **** THIS IS THE FINAL PRODUCT **** ####
import re

def add_false_answers_to_dataset(df: pd.DataFrame):  # INPUT DATAFRAME MUST CONTAIN COLUMNS OF CONTEXT, ANSWER AND QUESTION
  
  # ASSIGNING NAN VALUES TO EMPTY ROWS IN THE DATAFRAME:
  df['context'].replace('', np.nan, inplace=True)
  df['answer'].replace('', np.nan, inplace=True)
  df['question'].replace('', np.nan, inplace=True)
  # DROPPING THOSE ROWS:
  df = df.dropna(subset=['context', 'answer', 'question'])

  num_of_answers_generation = 20 # WE CHOOSE TO GENERATE 3 FALSE ANSWERS 20 DIFFERENT TIMES
  df["false_answer1"], df["false_answer2"], df["false_answer3"] = "", "", "" # ADDING CORRESPONDING COLLUMNS TO THE ORIGINAL DATAFRAME

  for index in range(len(df)): # RUNNING THROUGH THE DATAFRAME
    sample = df.iloc[index] # ALLOCATING A SINGLE SAMPLE FROM THE DATAFRAME
    real_answer, context, question = sample["answer"], sample["context"], sample["question"]
    real_answer = re.sub("""[!@#$%^â€“&*()_+=-]""", " ", real_answer) # PRE-PROCESSING THE REAL ANSWER SENTENCE, REMOVING SPECIAL CHARACTERS
    distilled_false_answers = [] # A LIST WHICH WILL EVENTUALLY CONTAIN 3 FALSE ANSWERS, AFTER PROCESSING (DISTILLATION)
    
    while len(distilled_false_answers) < 3: # WHILE WE STILL HAVEN'T CHOSEN 3 FALSE ANSWERS TO ACCOMPANY THE REAL ANSWER
      answer_word_count = len(real_answer.split()) # CALCULATING NUMBER OF WORDS IN THE REAL ANSWER
      generated = generate_preds(distractor_model, real_answer,
                  context, question, num_of_answers_generation)  # GENERATING THE 20 TRIOS OF FALSE ANSWERS
      #print(generated)
      false_answers = clean_prediction(generated) # CONVERTING THEM INTO A LIST OF FALSE ANSWERS
      #print(false_answers)
      for run in range(3): # 3 RUNS TO TRY CHOOSING A TOTAL OF 3 FALSE ANSWERS, 1ST RUN IS THE MOST STRICT, 3RD IS THE MOST LENIENT
        if len(distilled_false_answers) == 3: # WHEN WE ALREADY HAVE 3 FALSE ANSWERS THEN WE CAN STOP
          break
        for j in range(len(false_answers)): # RUNNING THROUGHT EVERY FALSE ANSWER AMONG THE 20*3 (=60) GENERATED ONES
          false_answer = false_answers[j]
          false_answer = re.sub("""[!@#$%^â€“&*()_+=-]""", " ", false_answer) # PRE-PROCESSING THE CURRENT FALSE ANSWER SENTENCE, REMOVING SPECIAL CHARACTERS
          if false_answer=='' or false_answer==' ':
            continue
          #print(false_answer)
          if answer_word_count <=3: # SHORTER ANSWERS DEMAND FALSE ANSWERS WITH HIGHER SIMILARITY THAN LONG ANSWERS
            limit_between_false_answers = 0.8 # DETERMINES THAT SIMILARITY BETWEEN FALSE ANSWERS WILL NOT EXCEED 0.8
            # NOW DEFINING SIMILARITY LIMITS BEFORE APPENDING FALSE ANSWERS:
            if run == 0: lower_limit, upper_limit = 0.6, 0.85 # FIRST RUN, STRICT
            if run == 1: lower_limit, upper_limit = 0.4, 0.6 # SECOND RUN, LESS STRICT
            if run == 2: lower_limit, upper_limit = 0, 0.4 # THIRD AND LAST RUN, VERY LENIENT, ALLOWS EVERY FALSE ANSWER NO MATTER THE SIMILARITIES
          else:
            limit_between_false_answers = 0.65 # DETERMINES THAT SIMILARITY BETWEEN FALSE ANSWERS WILL NOT EXCEED 0.65
            if run == 0: lower_limit, upper_limit = 0.5, 0.65 # FIRST RUN, STRICT
            if run == 1: lower_limit, upper_limit = 0.35, 0.5 # SECOND RUN, LESS STRICT
            if run == 2: lower_limit, upper_limit = 0, 0.35 # THIRD AND LAST RUN, VERY LENIENT, ALLOWS EVERY FALSE ANSWER NO MATTER THE SIMILARITIES
          
          if len(distilled_false_answers) == 2: # IF TWO FALSE ANSWERS ARE ALREADY APPENDED
            if ((real_answer.lower() in glove.vocab) and (distilled_false_answers[0].lower() in glove.vocab)
              and (distilled_false_answers[1].lower() in glove.vocab) and (false_answer.lower() in glove.vocab)): # MAKING SURE CURRENT, FIRSTL & SECOND FALSE ANSWERS AND REAL ANSWERS CAN BE EMBEDDED USING GLoVE, ALSO MEANING 1 WORD ANSWERS
              second_false_answer_embedding = glove[distilled_false_answers[1].lower()] # GLoVE EMBEDDING THE SECOND CHOSEN FALSE ANSWER
              first_false_answer_embedding = glove[distilled_false_answers[0].lower()] # GLoVE EMBEDDING THE FIRST CHOSEN FALSE ANSWER
              real_answer_embedding = glove[real_answer.lower()] # GLoVE EMBEDDING THE REAL ANSWER
              false_answer_embedding = glove[false_answer.lower()] # GLoVE EMBEDDING THE CURRENT FALSE ANSWER
            else: # IF NOT, THEN USE UNIVERSAL SENTENCE ENCODER
              second_false_answer_embedding = USE_model([distilled_false_answers[1]])[0]
              first_false_answer_embedding = USE_model([distilled_false_answers[0]])[0]
              false_answer_embedding = USE_model([false_answer])[0] 
              real_answer_embedding = USE_model([real_answer])[0] 

              similarity1 = cosine(false_answer_embedding, first_false_answer_embedding) # CALCULATING SIMILARITY BETWEEN CURRENT FALSE ANSWER AND THE FIRST ONE THAT WAS APPENDED EARLIER
              similarity2 = cosine(false_answer_embedding, second_false_answer_embedding) # CALCULATING SIMILARITY BETWEEN CURRENT FALSE ANSWER AND THE SECOND ONE THAT WAS APPENDED EARLIER
              similarity = cosine(false_answer_embedding, real_answer_embedding) # CALCULATING SIMILARITY BETWEEN THE CURRENT FALSE ANSWER AND THE REAL ANSWER
              if ((lower_limit <= similarity <= upper_limit) and (similarity1 <= limit_between_false_answers)
                and (similarity2 <= limit_between_false_answers)): # MAKING SURE THE THIRD FALSE ANSWER WON'T BE IDENTICAL TO THE OTHER TWO AND WILL BE QUITE SIMILAR TO THE REAL ANSWER
                distilled_false_answers.append(false_answer) # APPEND THE FALSE ANSWER, AND NOW LEN(DISTILLED) = 3
                break # FINISHED APPENDING 3 FALSE ANSWERS
              continue # IF THE CURRENT FALSE ANSWER WAS NOT APPENDED, THEN CONTINUE ITERATING OVER j, MEANING GOING TO THE NEXT GENERATED FALSE ANSWER 
          
          if len(distilled_false_answers) == 1: # IF ONE FALSE ANSWER IS ALREADY APPENDED
            if ((real_answer.lower() in glove.vocab) and (distilled_false_answers[0].lower() in glove.vocab)
              and (false_answer.lower() in glove.vocab)): # MAKING SURE CURRENT AND FIRST CHOSEN FALSE ANSWERS AND REAL ANSWERS CAN BE EMBEDDED USING GLOVE, ALSO MEANING 1 WORD ANSWERS
              first_false_answer_embedding = glove[distilled_false_answers[0].lower()] # GLoVE EMBEDDING THE FIRST CHOSEN FALSE ANSWER
              real_answer_embedding = glove[real_answer.lower()] # GLoVE EMBEDDING THE REAL ANSWER
              false_answer_embedding = glove[false_answer.lower()] # GLoVE EMBEDDING THE CURRENT FALSE ANSWER
            else: # IF NOT, THEN USE UNIVERSAL SENTENCE ENCODER
              first_false_answer_embedding = USE_model([distilled_false_answers[0]])[0]
              false_answer_embedding = USE_model([false_answer])[0] 
              real_answer_embedding = USE_model([real_answer])[0] 

            similarity1 = cosine(false_answer_embedding, first_false_answer_embedding) # CALCULATING SIMILARITY BETWEEN CURRENT FALSE ANSWER AND THE FIRST ONE THAT WAS APPENDED EARLIER
            similarity = cosine(false_answer_embedding, real_answer_embedding) # CALCULATING SIMILARITY BETWEEN THE CURRENT FALSE ANSWER AND THE REAL ANSWER
            if (lower_limit <= similarity <= upper_limit) and (similarity1 <= limit_between_false_answers): # MAKING SURE FALSE ANWERS ARE NOT IDENTICAL BUT YET QUITE SIMILAR TO THE REAL ANSWER
              distilled_false_answers.append(false_answer) # APPEND THE FALSE ANSWER, AND NOW LEN(DISTILLED) = 2
            continue # CONTINUE ITERATING OVER j, MEANING GOING TO THE NEXT GENERATED FALSE ANSWER 
          
          if len(distilled_false_answers) == 0: # IF NO FALSE ANSWERS WERE APPENDED YET
            if (real_answer.lower() in glove.vocab) and (false_answer.lower() in glove.vocab): # MAKING SURE CURRENT FALSE ANSWER AND REAL ANSWERS CAN BE EMBEDDED USING GLoVE, ALSO MEANING 1 WORD ANSWERS
              real_answer_embedding = glove[real_answer.lower()] # GLoVE EMBEDDING THE REAL ANSWER
              false_answer_embedding = glove[false_answer.lower()] # GLoVE EMBEDDING THE CURRENT FALSE ANSWER
            else: # IF NOT, THEN USE UNIVERSAL SENTENCE ENCODER
              false_answer_embedding = USE_model([false_answer])[0] 
              real_answer_embedding = USE_model([real_answer])[0] 
            similarity = cosine(false_answer_embedding, real_answer_embedding) # CALCULATING SIMILARITY BETWEEN THE CURRENT FALSE ANSWER AND THE REAL ANSWER
            if lower_limit <= similarity <= upper_limit: # MAKING SURE THEY ARE NOT IDENTICAL BUT YET QUITE SIMILAR
              distilled_false_answers.append(false_answer) # APPEND THE FALSE ANSWER, AND NOW LEN(DISTILLED) = 1

    # ASSIGNING THE FALSE ANSWERS TO THE DATAFRAME:
    #print("INDEX: ", index, distilled_false_answers)
    df.at[index,"false_answer1"] = distilled_false_answers[0]
    df.at[index,"false_answer2"] = distilled_false_answers[1]
    df.at[index,"false_answer3"] = distilled_false_answers[2]
    

  return df

In [93]:
try_df = pd.read_csv("ContextAnswerQuestion.csv")
generated_df = add_false_answers_to_dataset(try_df)
columns_titles = ["context","question","answer",'false_answer1',
       'false_answer2', 'false_answer3']
generated_df=generated_df.reindex(columns=columns_titles)
generated_df.to_csv('FinalOutput.csv', index=False,encoding='utf-8',)

In [94]:
generated_df

,context,question,answer,false_answer1,false_answer2,false_answer3
0,A much smaller number of people are insensitiv...,What is the inborn abnormality of a nervous sy...,congenital insensitivity to pain,cell spine sensitive to pain,a hereditary sensory and autonomic neuropathies,abnormal nervous system
1,A much smaller number of people are insensitiv...,What is the other side effect of congenital in...,a reduced life expectancy,a reduced disease,a lower age,a lower blood pressure
2,A much smaller number of people are insensitiv...,What does the SCN9A gene code for?,sodium channel (Nav1.7) necessary in conducti...,pain nerve stimuli,"a condition called ""congenital insensitivity ...",sensitivity to brain disorders
3,A much smaller number of people are insensitiv...,What are most people with congenital insensiti...,one of five hereditary sensory and autonomic ...,two of five sensory and motorneys,a one of three sensory and cognitive neuropathy,an inborn abnormalities of the nerve system
4,A much smaller number of people are insensitiv...,What is the name of a condition that affects c...,inflammation,abnormality,a neuropathy,AIDS
...,...,...,...,...,...,...
1198,"However, several de facto standards for tag fo...",When did ID3v1 and APEvi2 become more widespread?,2010,2011,2009,2008
1199,"However, several de facto standards for tag fo...",What are the most widespread standards for tag...,identification3v1 and ID3v2,ID3ev2 and ID0gebung.,ID3v1 and ID3w2.,"ID3,v2 and ID3.x2"
1200,"However, several de facto standards for tag fo...",What type of format exist in the United States?,Tag format,PM3 format,MP3 format,Decoder format
1201,"However, several de facto standards for tag fo...",When are tags normally embedded?,"at the beginning or end of MP3 files, separat...",at either in MP3 decoders or MP3,in the beginning of MP3.,"at the end or end, separate for MP3 frames"


In [114]:
context="""Bears are carnivoran mammals of the family Ursidae. They are classified as caniforms, or doglike carnivorans. Although only eight species of bears are extant, they are widespread, appearing in a wide variety of habitats throughout the Northern Hemisphere and partially in the Southern Hemisphere. Bears are found on the continents of North America, South America, Europe, and Asia. Common characteristics of modern bears include large bodies with stocky legs, long snouts, small rounded ears, shaggy hair, plantigrade paws with five nonretractile claws, and short tails While the polar bear is mostly carnivorous, and the giant panda feeds almost entirely on bamboo, the remaining six species are omnivorous with varied diets. With the exception of courting individuals and mothers with their young, bears are typically solitary animals. They may be diurnal or nocturnal and have an excellent sense of smell. Despite their heavy build and awkward gait, they are adept runners, climbers, and swimmers. Bears use shelters, such as caves and logs, as their dens; most species occupy their dens during the winter for a long period of hibernation, up to 100 days.Bears have been hunted since prehistoric times for their meat and fur; they have been used for bear-baiting and other forms of entertainment, such as being made to dance. With their powerful physical presence, they play a prominent role in the arts, mythology, and other cultural aspects of various human societies. In modern times, bears have come under pressure through encroachment on their habitats and illegal trade in bear parts, including the Asian bile bear market. The IUCN lists six bear species as vulnerable or endangered, and even least concern species, such as the brown bear, are at risk of extirpation in certain countries. The poaching and international trade of these most threatened populations are prohibited, but still ongoing. The English word bear comes from Old English bera and belongs to a family of names for the bear in Germanic languages, such as Swedish björn, also used as a first name. This form is conventionally said to be related to a Proto-Indo-European word for brown, so that bear would mean the brown one.[1][2] However, Ringe notes that while this etymology is semantically plausible, a word meaning brown of this form cannot be found in Proto-Indo-European. He suggests instead that bear is from the Proto-Indo-European word *ǵʰwḗr- ~ *ǵʰwér wild animal.[3] This terminology for the animal originated as a taboo avoidance term: proto-Germanic tribes replaced their original word for bear—arkto—with this euphemistic expression out of fear that speaking the animal's true name might cause it to appear.[4][5] According to author Ralph Keyes, this is the oldest known euphemism.[6] Bear taxon names such as Arctoidea and Helarctos come from the ancient Greek ἄρκτος (arktos), meaning bear,[7] as do the names arctic and antarctic, via the name of the constellation Ursa Major, the Great Bear, prominent in the northern sky.[8] Bear taxon names such as Ursidae and Ursus come from Latin Ursus/Ursa, he-bear/she-bear.[8] The female first name Ursula, originally derived from a Christian saint's name, means little she-bear (diminutive of Latin ursa). In Switzerland, the male first name Urs is especially popular, while the name of the canton and city of Bern is derived from Bär, German for bear. The Germanic name Bernard (including Bernhardt and similar forms)"""
answersList=GeneAnswersToListNormal(AnswerGen_model,context,500,10)
answersList



[' Ursidae',
 ' caniforms, or doglike carnivorans',
 ' the continents of North America, South America, Europe, and Asia',
 ' they are adept runners, climbers, and swimmers',
 ' bears have come under pressure through encroachment on their habitats and illegal trade in bear parts, including the Asian bile bear market.',
 ' The poaching and international trade of these animals']

In [111]:
context="""Bears are carnivoran mammals of the family Ursidae. They are classified as caniforms, or doglike carnivorans. Although only eight species of bears are extant, they are widespread, appearing in a wide variety of habitats throughout the Northern Hemisphere and partially in the Southern Hemisphere. Bears are found on the continents of North America, South America, Europe, and Asia. Common characteristics of modern bears include large bodies with stocky legs, long snouts, small rounded ears, shaggy hair, plantigrade paws with five nonretractile claws, and short tails While the polar bear is mostly carnivorous, and the giant panda feeds almost entirely on bamboo, the remaining six species are omnivorous with varied diets. With the exception of courting individuals and mothers with their young, bears are typically solitary animals. They may be diurnal or nocturnal and have an excellent sense of smell. Despite their heavy build and awkward gait, they are adept runners, climbers, and swimmers. Bears use shelters, such as caves and logs, as their dens; most species occupy their dens during the winter for a long period of hibernation, up to 100 days.Bears have been hunted since prehistoric times for their meat and fur; they have been used for bear-baiting and other forms of entertainment, such as being made to dance. With their powerful physical presence, they play a prominent role in the arts, mythology, and other cultural aspects of various human societies. In modern times, bears have come under pressure through encroachment on their habitats and illegal trade in bear parts, including the Asian bile bear market. The IUCN lists six bear species as vulnerable or endangered, and even least concern species, such as the brown bear, are at risk of extirpation in certain countries. The poaching and international trade of these most threatened populations are prohibited, but still ongoing. The English word bear comes from Old English bera and belongs to a family of names for the bear in Germanic languages, such as Swedish björn, also used as a first name. This form is conventionally said to be related to a Proto-Indo-European word for brown, so that bear would mean the brown one.[1][2] However, Ringe notes that while this etymology is semantically plausible, a word meaning brown of this form cannot be found in Proto-Indo-European. He suggests instead that bear is from the Proto-Indo-European word *ǵʰwḗr- ~ *ǵʰwér wild animal.[3] This terminology for the animal originated as a taboo avoidance term: proto-Germanic tribes replaced their original word for bear—arkto—with this euphemistic expression out of fear that speaking the animal's true name might cause it to appear.[4][5] According to author Ralph Keyes, this is the oldest known euphemism.[6] Bear taxon names such as Arctoidea and Helarctos come from the ancient Greek ἄρκτος (arktos), meaning bear,[7] as do the names arctic and antarctic, via the name of the constellation Ursa Major, the Great Bear, prominent in the northern sky.[8] Bear taxon names such as Ursidae and Ursus come from Latin Ursus/Ursa, he-bear/she-bear.[8] The female first name Ursula, originally derived from a Christian saint's name, means little she-bear (diminutive of Latin ursa). In Switzerland, the male first name Urs is especially popular, while the name of the canton and city of Bern is derived from Bär, German for bear. The Germanic name Bernard (including Bernhardt and similar forms)"""
data=[]
data.append(context)
singleAnswer_df=pd.DataFrame(data=data,columns=['context'])

contextAnswer_df=GeneAnswersForDFToDF(AnswerGen_model,singleAnswer_df,500)
contextAnswer_df


,context,answer
0,Bears are carnivoran mammals of the family Urs...,"caniforms, or doglike carnivorans"
1,Bears are carnivoran mammals of the family Urs...,"the continents of North America, South Americ..."
2,Bears are carnivoran mammals of the family Urs...,Ursidae
3,Bears are carnivoran mammals of the family Urs...,bears have come under pressure through encroa...
4,Bears are carnivoran mammals of the family Urs...,The poaching and international trade of these...
5,Bears are carnivoran mammals of the family Urs...,"they are adept runners, climbers, and swimmers"


In [ ]:
df = ans_text(context_ans)
columns_titles = ["context","question","answer"]
df=df.reindex(columns=columns_titles)

df.to_csv('ContextAnswerQuestion.csv',encoding='utf-8', index=False)

# Generate MCQ based on a given manual context 
generating up to 10 answers per context

In [120]:
context="""Bears are carnivoran mammals of the family Ursidae. They are classified as caniforms, or doglike carnivorans. Although only eight species of bears are extant, they are widespread, appearing in a wide variety of habitats throughout the Northern Hemisphere and partially in the Southern Hemisphere. Bears are found on the continents of North America, South America, Europe, and Asia. Common characteristics of modern bears include large bodies with stocky legs, long snouts, small rounded ears, shaggy hair, plantigrade paws with five nonretractile claws, and short tails While the polar bear is mostly carnivorous, and the giant panda feeds almost entirely on bamboo, the remaining six species are omnivorous with varied diets. With the exception of courting individuals and mothers with their young, bears are typically solitary animals. They may be diurnal or nocturnal and have an excellent sense of smell. Despite their heavy build and awkward gait, they are adept runners, climbers, and swimmers. Bears use shelters, such as caves and logs, as their dens; most species occupy their dens during the winter for a long period of hibernation, up to 100 days.Bears have been hunted since prehistoric times for their meat and fur; they have been used for bear-baiting and other forms of entertainment, such as being made to dance. With their powerful physical presence, they play a prominent role in the arts, mythology, and other cultural aspects of various human societies. In modern times, bears have come under pressure through encroachment on their habitats and illegal trade in bear parts, including the Asian bile bear market. The IUCN lists six bear species as vulnerable or endangered, and even least concern species, such as the brown bear, are at risk of extirpation in certain countries. The poaching and international trade of these most threatened populations are prohibited, but still ongoing. The English word bear comes from Old English bera and belongs to a family of names for the bear in Germanic languages, such as Swedish björn, also used as a first name. This form is conventionally said to be related to a Proto-Indo-European word for brown, so that bear would mean the brown one.[1][2] However, Ringe notes that while this etymology is semantically plausible, a word meaning brown of this form cannot be found in Proto-Indo-European. He suggests instead that bear is from the Proto-Indo-European word *ǵʰwḗr- ~ *ǵʰwér wild animal.[3] This terminology for the animal originated as a taboo avoidance term: proto-Germanic tribes replaced their original word for bear—arkto—with this euphemistic expression out of fear that speaking the animal's true name might cause it to appear.[4][5] According to author Ralph Keyes, this is the oldest known euphemism.[6] Bear taxon names such as Arctoidea and Helarctos come from the ancient Greek ἄρκτος (arktos), meaning bear,[7] as do the names arctic and antarctic, via the name of the constellation Ursa Major, the Great Bear, prominent in the northern sky.[8] Bear taxon names such as Ursidae and Ursus come from Latin Ursus/Ursa, he-bear/she-bear.[8] The female first name Ursula, originally derived from a Christian saint's name, means little she-bear (diminutive of Latin ursa). In Switzerland, the male first name Urs is especially popular, while the name of the canton and city of Bern is derived from Bär, German for bear. The Germanic name Bernard (including Bernhardt and similar forms)"""
answersList=GeneAnswersToListNormal(AnswerGen_model,context,500,10)
#print(answersList)
data=[]
for answer in answersList:
  data.append([context,answer])
answersContextDataFrame=pd.DataFrame(data=data,columns=['context','answer'])
answersContextDataFrame.to_csv('SingleAnswer.csv',encoding='utf-8', index=False)



context_ans = pd.read_csv('SingleAnswer.csv')
df = ans_text(context_ans)
columns_titles = ["context","question","answer"]
df=df.reindex(columns=columns_titles)
df.to_csv('SingleAnswerQuestion.csv',encoding='utf-8', index=False)



try_df = pd.read_csv("SingleAnswerQuestion.csv")
generated_df = add_false_answers_to_dataset(try_df)
columns_titles = ["context","question","answer",'false_answer1',
       'false_answer2', 'false_answer3']
generated_df=generated_df.reindex(columns=columns_titles)
generated_df.to_csv('FinalOutputSingle.csv', index=False,encoding='utf-8',)


[' Ursidae', ' caniforms, or doglike carnivorans', ' the continents of North America, South America, Europe, and Asia', ' they are adept runners, climbers, and swimmers', ' bears have come under pressure through encroachment on their habitats and illegal trade in bear parts, including the Asian bile bear market.', ' The poaching and international trade of these animals']


# **Generating an exact amount of questions per context** 

In [150]:

context="""Bears are carnivoran mammals of the family Ursidae. They are classified as caniforms, or doglike carnivorans. Although only eight species of bears are extant, they are widespread, appearing in a wide variety of habitats throughout the Northern Hemisphere and partially in the Southern Hemisphere. Bears are found on the continents of North America, South America, Europe, and Asia. Common characteristics of modern bears include large bodies with stocky legs, long snouts, small rounded ears, shaggy hair, plantigrade paws with five nonretractile claws, and short tails While the polar bear is mostly carnivorous, and the giant panda feeds almost entirely on bamboo, the remaining six species are omnivorous with varied diets. With the exception of courting individuals and mothers with their young, bears are typically solitary animals. They may be diurnal or nocturnal and have an excellent sense of smell. Despite their heavy build and awkward gait, they are adept runners, climbers, and swimmers. Bears use shelters, such as caves and logs, as their dens; most species occupy their dens during the winter for a long period of hibernation, up to 100 days.Bears have been hunted since prehistoric times for their meat and fur; they have been used for bear-baiting and other forms of entertainment, such as being made to dance. With their powerful physical presence, they play a prominent role in the arts, mythology, and other cultural aspects of various human societies. In modern times, bears have come under pressure through encroachment on their habitats and illegal trade in bear parts, including the Asian bile bear market. The IUCN lists six bear species as vulnerable or endangered, and even least concern species, such as the brown bear, are at risk of extirpation in certain countries. The poaching and international trade of these most threatened populations are prohibited, but still ongoing. The English word bear comes from Old English bera and belongs to a family of names for the bear in Germanic languages, such as Swedish björn, also used as a first name. This form is conventionally said to be related to a Proto-Indo-European word for brown, so that bear would mean the brown one.[1][2] However, Ringe notes that while this etymology is semantically plausible, a word meaning brown of this form cannot be found in Proto-Indo-European. He suggests instead that bear is from the Proto-Indo-European word *ǵʰwḗr- ~ *ǵʰwér wild animal.[3] This terminology for the animal originated as a taboo avoidance term: proto-Germanic tribes replaced their original word for bear—arkto—with this euphemistic expression out of fear that speaking the animal's true name might cause it to appear.[4][5] According to author Ralph Keyes, this is the oldest known euphemism.[6] Bear taxon names such as Arctoidea and Helarctos come from the ancient Greek ἄρκτος (arktos), meaning bear,[7] as do the names arctic and antarctic, via the name of the constellation Ursa Major, the Great Bear, prominent in the northern sky.[8] Bear taxon names such as Ursidae and Ursus come from Latin Ursus/Ursa, he-bear/she-bear.[8] The female first name Ursula, originally derived from a Christian saint's name, means little she-bear (diminutive of Latin ursa). In Switzerland, the male first name Urs is especially popular, while the name of the canton and city of Bern is derived from Bär, German for bear. The Germanic name Bernard (including Bernhardt and similar forms)"""

context="""A robot is a machine—especially one programmable by a computer—capable of carrying out a complex series of actions automatically.[2] A robot can be guided by an external control device, or the control may be embedded within. Robots may be constructed to evoke human form, but most robots are task-performing machines, designed with an emphasis on stark functionality, rather than expressive aesthetics.

Robots can be autonomous or semi-autonomous and range from humanoids such as Honda's Advanced Step in Innovative Mobility (ASIMO) and TOSY's TOSY Ping Pong Playing Robot (TOPIO) to industrial robots, medical operating robots, patient assist robots, dog therapy robots, collectively programmed swarm robots, UAV drones such as General Atomics MQ-1 Predator, and even microscopic nano robots. By mimicking a lifelike appearance or automating movements, a robot may convey a sense of intelligence or thought of its own. Autonomous things are expected to proliferate in the future, with home robotics and the autonomous car as some of the main drivers.[3]

The branch of technology that deals with the design, construction, operation, and application of robots,[4] as well as computer systems for their control, sensory feedback, and information processing is robotics. These technologies deal with automated machines that can take the place of humans in dangerous environments or manufacturing processes, or resemble humans in appearance, behavior, or cognition. Many of today's robots are inspired by nature contributing to the field of bio-inspired robotics. These robots have also created a newer branch of robotics: soft robotics.

From the time of ancient civilization, there have been many accounts of user-configurable automated devices and even automata resembling humans and other animals, designed primarily as entertainment. As mechanical techniques developed through the Industrial age, there appeared more practical applications such as automated machines, remote-control and wireless remote-control.

The term comes from a Slavic root, robot-, with meanings associated with labor. The word 'robot' was first used to denote a fictional humanoid in a 1920 Czech-language play R.U.R. (Rossumovi Univerzální Roboti – Rossum's Universal Robots) by Karel Čapek, though it was Karel's brother Josef Čapek who was the word's true inventor.[5][6][7] Electronics evolved into the driving force of development with the advent of the first electronic autonomous robots created by William Grey Walter in Bristol, England in 1948, as well as Computer Numerical Control (CNC) machine tools in the late 1940s by John T. Parsons and Frank L. Stulen.

The first modern digital and programmable robot was invented by George Devol in 1954 and spawned his seminal robotics company, Unimation. The first Unimate was sold to General Motors in 1961 where it lifted pieces of hot metal from die casting machines at the Inland Fisher Guide Plant in the West Trenton section of Ewing Township, New Jersey.[8]

Robots have replaced humans[9] in performing repetitive and dangerous tasks which humans prefer not to do, or are unable to do because of size limitations, or which take place in extreme environments such as outer space or the bottom of the sea. There are concerns about the increasing use of robots and their role in society. Robots are blamed for rising technological unemployment as they replace workers in increasing numbers of functions.[10] The use of robots in military combat raises ethical concerns. The possibilities of robot autonomy and potential repercussions have been addressed in fiction and may be a realistic concern in the future."""

data=[]
data.append(context)
singleAnswer_df=pd.DataFrame(data=data,columns=['context'])

answersContextDataFrame=GeneAnswersForDFToDFWithMaxAnswers(AnswerGen_model,singleAnswer_df,500,20,1)


answersContextDataFrame=pd.DataFrame(data=answersContextDataFrame,columns=['context','answer'])
answersContextDataFrame.to_csv('SingleAnswer.csv',encoding='utf-8', index=False)


context_ans = pd.read_csv('SingleAnswer.csv')
df = ans_text(context_ans)
columns_titles = ["context","question","answer"]
df=df.reindex(columns=columns_titles)
df.to_csv('SingleAnswerQuestion.csv',encoding='utf-8', index=False)



try_df = pd.read_csv("SingleAnswerQuestion.csv")
generated_df = add_false_answers_to_dataset(try_df)
columns_titles = ["context","question","answer",'false_answer1',
       'false_answer2', 'false_answer3']
generated_df=generated_df.reindex(columns=columns_titles)
generated_df.to_csv('FinalOutputSingle.csv', index=False,encoding='utf-8',)
generated_df

,context,question,answer,false_answer1,false_answer2,false_answer3
0,A robot is a machine—especially one programmab...,What do robots have to be constructed for?,evoke human form,mimic the appearance of people,exhibit lifelike appearance,stimulate human behavior
1,A robot is a machine—especially one programmab...,What is the name of a new branch in robotics?,"""soft robotics.""",robot .,"""uncontrolled robotics.""","""bio inspired robotics."""
2,A robot is a machine—especially one programmab...,What is the name of a UAV drone?,MQ-1 Predator,MQ 1 Robot,TOSY Pong Playing Robot,ATIMO
3,A robot is a machine—especially one programmab...,What is the term robot?,Slavic root,Robot,Uva,Unable
4,A robot is a machine—especially one programmab...,What type of tasks are robots able to perform?,repetitive repetitive repetitive tasks,autonomous tasks,expressive aesthetics,tasks that require some attention
5,A robot is a machine—especially one programmab...,What is the main function of a robot?,the control may be embedded within.,An external control device may be employed.,The control may be built by robots.,its engine may be controlled.
6,A robot is a machine—especially one programmab...,What is the name of a Czech-language play?,robots,robots of the same name,robot of the last,robot's inventions
7,A robot is a machine—especially one programmab...,What is the name of a robot that can carry out...,AI,Robot,A robot,robotic robot
8,A robot is a machine—especially one programmab...,What does a robot convey by mimicking lifelike...,artificial intelligence,a feeling of intelligence,emotional intelligence,intelligence
9,A robot is a machine—especially one programmab...,What is the name of a robot that can take plac...,automation,robotic robotics,robots,machine
